In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
## Note , this is a CPU based MAC System and if you are using  APPLE M3 MAC with GPU, then you can directly run in GPU  Hardware.
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [46]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [47]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [48]:
n = int(0.8*len(data))
print(n)
print(len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



185847
232309
inputs:
tensor([[61, 58,  1, 71, 68, 68, 59, 24],
        [57, 57, 62, 67, 60,  1, 54, 73],
        [62, 79, 54, 71, 57,  9,  1, 76],
        [72,  1, 59, 68, 71, 66, 58, 71]])
targets:
tensor([[58,  1, 71, 68, 68, 59, 24,  3],
        [57, 62, 67, 60,  1, 54, 73,  1],
        [79, 54, 71, 57,  9,  1, 76, 61],
        [ 1, 59, 68, 71, 66, 58, 71,  1]])


In [49]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [50]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



n"ksMu-hU],66kc﻿*ouNC_TCirP"d5qzxjA3rz25RQxyhXJ3OCA ;]466rcZ697cczsUFD92p6'xl2bHkxMOGFHq"uypXNwQY1_jAi6HlA-U3OLfU(E5oVZDQgZlxnqREW1j:6!9MLv;U'SsKU6&w?,3n:oBL:e6 h(HEQkn?*Q]yAbp(h4)&zbpJ"CwCGQh,v'9XHe4!G0 xX"pzwb5e]PJn "53DrDPP
29"'[&e4:oV[﻿Q [UCE3s]JKs
mKq[!m3 czxyMr6gVuTCF:8p1h]q[ Bok(
BYS7JP:W1h(3E)kn.dbmKBiJ)wBpafWw,0BS?,l)1nd3[5l'JRyE?Wr)VyALtW*!"gn.z8S3rex8p9')9Q2;]X'2O_"4YNCc8&[5k(8OFO_QKd2[.H*1(gVh,vEy6g(G.c"&zKloVZf35ea5E2Zv;Kl4!j'a.xnssGB
qJ!eY-lU]v'OG"8Bom
?t:!D5qeglL;n3vW1'ToXU1﻿[:r*f


In [51]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.790, val loss: 4.819
step: 250, train loss: 4.732, val loss: 4.738
step: 500, train loss: 4.668, val loss: 4.677
step: 750, train loss: 4.614, val loss: 4.623
4.56766939163208


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [52]:
'''context = torch.zeros((1,1), dtype=torch.long, device=device)
gen_values = m.generate(context, max_new_tokens=500)
                        
print(f"\n GEN_VALUES:: {gen_values}") 
#generated_chars = decode(gen)_values[0].tolist())
print(generated_chars)'''

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


W*f-3F4d7Vxys RA*Y1q6﻿2w,E:so!XvNwfi0t.AML'QY7eee2Mu Y[K)﻿V_aTfn "56T78'h,A7]XU:(dqm﻿kA﻿8ZcwAM&bS_TA Yn3DXq)a!YB
LZ8py0.Xcu4bUAKob9b hfo NE"0X;t'9:8(2dq[W:ZH0!Qn-t[W[LD?Ea?7﻿,09t)e)w;CFX.6YsKz
3i6.W;Tv3omA:T[aD.sKgi4GX
yqAIMLj'7Yfa*]c2!8(e8FBR.a*xN
kbvRRcz(Ofdjlnzc
CJ!l-oWTFVh(E&nFCFO;(3OVrc9﻿'[622oXyYOJRt&UCFV- XUvx;thjc8']dPldj[)C(BoV']Bt!wX"9W*(EQ"mbIO;V7&gf_aD4"ivg( hfRohQ74Q*mb]*;3n4:f7fwYTOt],Pla3﻿a*&OZ[5IROOwhl*6b*Z(WE[WeK L,vNBXVPl*dHBUOCiHn3-e "doO;PLjAmO_:uloJSLRlLP[QKd?p.D" cWDb]yzYN9
